In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import sys


In [1]:
import glob 
import os
import re
import pandas as pd

In [2]:
def to_scientific(value):
    return float(f"{value:.2e}")

In [5]:
exact = True
fd_comparative = ["correlation", "sisj", "sisjsk", "triplet", "magnetization"]
if(exact==True):
    folder_comp = "../Results/Comparative/"
else:
    folder_comp = "../Results_Metropolis/Comparative/"

